# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
# Read the file 
weather_file = "output_data/city_weather_data.csv"
weather_data = pd.read_csv(weather_file)

# Put it in a DataFrame
weatherdf = pd.DataFrame(weather_data)
weatherdf.head()

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,east london,-33.02,27.91,67.08,87.0,99.0,21.32,ZA,1.603755e+09
1,malpe,13.35,74.72,77.00,88.0,20.0,2.80,IN,1.603755e+09
2,hofn,64.25,-15.21,36.54,78.0,4.0,10.74,IS,1.603755e+09
3,sulangan,11.14,123.72,82.63,75.0,100.0,10.22,PH,1.603755e+09
4,butaritari,3.07,172.79,81.72,67.0,77.0,9.82,KI,1.603755e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [11]:
# Store 'Lat' and 'Lng' into  locations 
locations = weatherdf[["Lat", "lng"]].astype(float)

#weatherdf = weatherdf.dropna()

hum_rate = weatherdf["Humidity"].astype(float)


In [12]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
#Finding the ideal weather condition.

#A max temperature lower than 80 degrees but higher than 70.
filtered_weather = weatherdf.loc[weatherdf["Max Temp"]<80,:]
filtered_weather = first.loc[weatherdf["Max Temp"]>70,:]

#Wind speed less than 10 mph.
filtered_weather = second.loc[second["Wind Speed"]<10,:]

#Zero cloudiness.
filtered_weather = third.loc[third["Cloudiness"]==0,:]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = pd.DataFrame()
filtered_weather = pd.DataFrame(filtered_weather)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Lat']
hotel_df['Lng'] = fifth['lng']
#hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng
99,arlit,NE,18.74,7.39
103,tirat karmel,IL,32.76,34.97
279,salalah,OM,17.02,54.09
308,nandura,IN,20.83,76.45
332,islamkot,PK,24.70,70.18


In [33]:
# use iterrow  to get the dataframe 
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 99: arlit.
Closest hotel to arlit at 18.74 , 7.39 is Trakou Marche.
------------
Retrieving Results for Index 103: tirat karmel.
Closest hotel to tirat karmel at 32.76 , 34.97 is Garden Hotel.
------------
Retrieving Results for Index 279: salalah.
Closest hotel to salalah at 17.02 , 54.09 is Alrehab Palace 1.
------------
Retrieving Results for Index 308: nandura.
Closest hotel to nandura at 20.83 , 76.45 is Hanuman And Shree Balaji Mandir.
------------
Retrieving Results for Index 332: islamkot.
Closest hotel to islamkot at 24.7 , 70.18 is Town Committee Old Shopping Center.
------------
Retrieving Results for Index 402: keti bandar.
Closest hotel to keti bandar at 24.14 , 67.45 is Nawaz Khan Crab Company.
------------
Retrieving Results for Index 437: inhambane.
Closest hotel to inhambane at -23.86 , 35.38 is Saboeira.
------------
Retrieving Results for Index 440: koungou.
Closest hotel to koungou at -12.73 , 45.2 is Selectour - AMVoyages.
------------


In [34]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
99,arlit,NE,18.74,7.39,Trakou Marche
103,tirat karmel,IL,32.76,34.97,Garden Hotel
279,salalah,OM,17.02,54.09,Alrehab Palace 1
308,nandura,IN,20.83,76.45,Hanuman And Shree Balaji Mandir
332,islamkot,PK,24.70,70.18,Town Committee Old Shopping Center


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

In [46]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations,  
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)


hotel_layer = gmaps.marker_layer(locations2, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

plt.savefig("vacationheatmapwithhotels.png")

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>